# Imports

In [ ]:
import os
import sys
import time
import re
import pickle
import logging
import string
import warnings
import math

import pandas as pd
import numpy as np
import pylab
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

import nltk
import nltk.data
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.cluster import KMeans
from sklearn.metrics import roc_auc_score as AUC

import gensim
from gensim import corpora
from gensim import models
from gensim import similarities
from gensim.models import Word2Vec

from bs4 import BeautifulSoup
from sklearn.metrics import log_loss

# Get Data

In [ ]:
nltk.download("stopwords")
quora_train = pd.read_csv("data/train.csv")

In [ ]:
print (type(quora_train))
print(quora_train.head())

# Functions to process data

In [ ]:
# Editing questions with NLTK package

def remove_stopwords(phrase,list_stopwords):
    
    final_phrase = []
    words = phrase.split(" ")
    for word in words:
        if word not in list_stopwords:
            final_phrase.append((word))
    
    final_phrase = ' '.join(final_phrase)
    
    return final_phrase
    
def remove_punctuation(phrase):
    
    #Check if NA
    if type(phrase) is float:
        if math.isnan(phrase):
            return ("")
    
    translator = str.maketrans('', '', string.punctuation)
    phrase = phrase.translate(translator) #removing punctuation
        
    return phrase

def lemm_wordnet(phrase):
    """
    Lemmatize using WordNet’s built-in morphy function. 
    Returns the input word unchanged if it cannot be found in WordNet
    """
    lemm = WordNetLemmatizer()
    
    #NA is a float type, so this if is to avoid conflict
    if type(phrase) is not float:
        phrase = [lemm.lemmatize(i) for i in phrase.split()]
        phrase = ' '.join(phrase)
    else:
        return ""
    return phrase
    

def stem_stopwords_punctuation_case (phrase):
    """
    Receives a phrase and returns the same phrase stemmed, lowercase phrase without stopwords
    :param package_name: String. A phrase.
    :return: String. Stemmed, lowercase phrase without stopwords
    """
    stopwords = nltk.corpus.stopwords.words('english')
    stemmer = SnowballStemmer("english")
    
    phrase = remove_punctuation(phrase)
    phrase = phrase.lower() #removing word case
    
    

    
    phrase = remove_stopwords(phrase,stopwords)
    
    #Stem words according to stemmer
    final_phrase = []
    words = phrase.split(" ")
    for word in words:
        final_phrase.append((stemmer.stem(word)))
    
    final_phrase = ' '.join(final_phrase)
    
    return final_phrase

stem_stopwords_punctuation_case(
    "What is the step by step guide to invest in share market in india?")



In [ ]:
quora_train["question1_edited"] = quora_train["question1"].apply(lambda x: stem_stopwords_punctuation_case(x))
quora_train["question2_edited"] = quora_train["question2"].apply(lambda x: stem_stopwords_punctuation_case(x))

print (quora_train.head())

# Store edited databases w/ Pickle

In [ ]:
fileObject = open("Edited_Base_stem_stopwords",'wb') 
pickle.dump(quora_train,fileObject)  
fileObject.close()

In [ ]:
fileObject = open("Edited_Base_stem_stopwords",'rb')  
quora_train = pickle.load(fileObject)

# Exploratory Data Analysis

In [ ]:
def calculate_common_percentage(df):
    """
    Receives the initial data frame and adds  the colunms "num_words_common", "num_words_total" and "common_percentage"
    :param package_name: Data frame train.csv from the Kaggle website
    :return: Data frame with added colunms "num_words_common", "num_words_total" and "common_percentage"
    """

    num_words_common = []
    num_words_total = []

    for line in range(0,len(df)):
        count_total = 0
        count_common = 0
        for word in df["question1_edited"][line].split(" "):
            if word in df["question2_edited"][line]:
                count_common = count_common+1
            count_total = count_total+1
        num_words_common.append(count_common) 
        num_words_total.append(count_total)

    num_words_common = pd.Series(num_words_common)
    num_words_total = pd.Series(num_words_total)

    df["num_words_common"] = num_words_common.values
    df["num_words_total"] = num_words_total.values
    df["common_percentage"] = df["num_words_common"]/df["num_words_total"]

    return (df)



In [ ]:
quora_train = calculate_common_percentage(quora_train)
#"num_words_common", "num_words_total" and "common_percentage"
plt.figure()
#plt.boxplot(quora_train["common_percentage"],quora_train["is_duplicate"])


quora_train.boxplot(column='common_percentage', by='is_duplicate')
plt.suptitle('')

axes = plt.gca()
axes.set_ylim([-0.2,1.1])
plt.title("Common word percentage and is_duplicate")

plt.show()

# Learning Models